<h1> Automate Scrapping <a href="https://myanimelist.net/topanime.php" target="_blank">MyAnimelist</a> website using Python </h1>
by: Hafizh Iman Naufal

<h4 style="background-color:tomato;"></h4>

<h2> How to Use? </h2>
<p style='text-align: justify;'> 1. Define the top anime page, which order of the top anime should i scrap?
 there are 10 type of order you can choose:</p>

* 'allanime' = top anime of all time
* 'bypopularity' = top anime sorted by popularity
* 'favorite' = top anime sorted by most favorites
* 'airing' = top airing anime
* 'upcoming' = top upcoming anime
* 'tv' = top anime TV series
* 'movie' = top anime movies
* 'ova' = top OVA anime
* 'ona' = top ONA anime
* 'special' = top special anime


<p style='text-align: justify;'> 2. the default values of the page anime that will be gathered is 100 (from rank 1 to 100), you can change this value by defining the input (top_min top_max) </p>


`Any comments or suggestion are welcome, thanks :)`


In [7]:
from requests_html import HTMLSession
from requests_html import AsyncHTMLSession
import numpy as np
from bs4 import BeautifulSoup
import re
import os.path
import httpx
import asyncio
import nest_asyncio
import aiohttp
from lxml.html.soupparser import fromstring
import urllib.request as urllib2
import bs4
import time
from bs4 import BeautifulSoup, NavigableString



# calculate how many anime should be scrap on each pages
def pages(x):
    if x < 50:
        return 0
    elif x % 50 == 0:
        return x - 50
    else:
        return (x + 50 - x % 50) - 50

# set async coroutine
nest_asyncio.apply()

async def main(url):
    s, init = 0, True
    while init:
        try:
            time.sleep(s)
            async with aiohttp.ClientSession() as session:
                async with session.get(url) as r:
                    teks = await r.text()
                    print(f'{url} - {r.status}')
            init = False
        except Exception as err:
            print(f'failed! - {err}-')
            s += 10
            print('try to fixing it! - {s}s')
            init = True
    return teks

class Scrapping_top_mal:

    def __init__(self, tipe, top_max, top_min = 1):
        self.top_min = top_min
        self.top_max = top_max
        self.tipe = tipe
     
    # get the top anime link
    def hal_top_anime(self):
        hal_top = np.arange(pages(self.top_min), pages(self.top_max) + 50, 50)
        for i in hal_top:
            yield f"https://myanimelist.net/topanime.php?type={self.tipe}&limit={i}"
    
    # scrap to get page link for each anime
    def get_link_anime(self):
        num = self.top_min
        for url_top in self.hal_top_anime():
            b_soup = BeautifulSoup(asyncio.run(main(url_top)),'html.parser')
            links = b_soup.find_all('a',class_ = re.compile('hoverinfo_trigger.*'), id = re.compile('\#area\d+'), rel = re.compile('\#info\d+'))
            for link in links:
                if self.top_min <= num <= self.top_max:
                    yield (num,BeautifulSoup(asyncio.run(main(link.get('href'))),'html.parser'),link.get('href'))
                num += 1
    
    # parsing anime page with beautiful soup
    def get_var(self):
        df = []
        souped = {}
        dup = lambda d:d[0:int(len(d)/2)]
        for content in self.get_link_anime():
            # get variables
            scrap = ''.join([pad.text for pad in content[1].find_all('div',{'class' : 'spaceit_pad'})])
            title = content[1].find('h1', class_ ='title-name h1_bold_none').text
            title_alt = re.findall(r'(.*)Types?:',re.sub(r'\n','',scrap)) if re.search(r'Types?:',scrap) else "N/A" 
            link = content[2]
            types = re.findall(r'\nTypes?:\n(.*)\n',scrap) if re.search(r'Types?:',scrap) else "N/A" 
            episode = re.findall(r'\nEpisodes?:\n(.*)\n',scrap) if re.search(r'Episodes?:',scrap) else "N/A" 
            aired = re.findall(r'\nAired:\n(.*)\n',scrap) if re.search(r'Aired:',scrap) else "N/A" 
            premiered = re.findall(r'\nPremiered:\n(.*)\n',scrap) if re.search(r'Premiered:',scrap) else "N/A"
            broadcast = re.findall(r'\nBroadcast:\n(.*)\n',scrap) if re.search(r'Broadcast:',scrap) else "N/A"
            producers = re.findall(r'\nProducers?:\n(.*)\n',scrap) if re.search(r'Producers?:',scrap) else "N/A"
            licensors = re.findall(r'\nLicensors?:\n(.*)\n',scrap) if re.search(r'Licensors?:',scrap) else "N/A"
            studios = re.findall(r'\nStudios?:\n(.*)\n',scrap) if re.search(r'Studios?:',scrap) else "N/A"
            source = re.findall(r'\nSources?:\n(.*)\n',scrap) if re.search(r'Sources?:',scrap) else "N/A"
            genre = re.findall(r'\nGenres?:\n(.*)\n',scrap) if re.search(r'Genres?:',scrap) else "N/A"
            genres = re.sub(r'\s{1,}','',''.join(genre))
            genres = [dup(x) for x in genres.split(',')]
            theme = re.findall(r'\nThemes?:\n(.*)\n',scrap) if re.search(r'Themes?:',scrap) else "N/A"
            themes = re.sub(r'\s{1,}','',''.join(theme))
            themes = [dup(x) for x in themes.split(',')]
            duration = re.findall(r'\nDuration:\n(.*)\n',scrap) if re.search(r'Duration:',scrap) else "N/A"
            rating = re.findall(r'\nRatings?:\n(.*)\n',scrap) if re.search(r'Ratings?:',scrap) else "N/A"
            rating_count = content[1].find('span',itemprop ='ratingCount')
            rating_counts = rating_count.text if rating_count else "N/A"
            scores = re.findall(r'\nScores?:\n(.*)\n',scrap) if re.search(r'Scores?:',scrap) else "N/A"
            ranked = re.findall(r'\nRanked:\n(.*)\n',scrap) if re.search(r'Ranked:',scrap) else "N/A"
            popularity = re.findall(r'\nPopularity:\n(.*)\n',scrap) if re.search(r'Popularity:',scrap) else "N/A"
            members = re.findall(r'\nMembers?:\n(.*)\n',scrap) if re.search(r'Members?:',scrap) else "N/A"
            favorites = re.findall(r'\nFavorites?:\n(.*)\n',scrap) if re.search(r'Favorites?:',scrap) else "N/A"
            synopsi = content[1].find('p',itemprop ='description')
            synopsis = synopsi.text if synopsi else 'N/A'
            # variables to dict
            data_mal = {
                'Title' : re.sub(r'\s{2,}',' ',title).strip() if title else "N/A",
                'Title_alt' : title_alt[0],
                'Link' : link,
                'Type' : re.sub(r'\s{2,}',' ',types[0]).strip(),
                'Episodes' : re.sub(r'\s{2,}',' ',episode[0]).strip(),
                'Aired' : re.sub(r'\s{2,}',' ',aired[0]).strip(),
                'Premiered' : re.sub(r'\s{2,}',' ',premiered[0]).strip(),
                'Broadcast' : re.sub(r'\s{2,}',' ',broadcast[0]).strip(),
                'Producers' : re.sub(r'\s{2,}',' ',producers[0]).strip(),
                'Licensors' : re.sub(r'\s{2,}',' ',licensors[0]).strip(),
                'Studios' : re.sub(r'\s{2,}',' ',studios[0]).strip(),
                'Source' : re.sub(r'\s{2,}',' ',source[0]).strip(),
                'Genres' : ','.join(genres),
                'Themes' : ','.join(themes),
                'Duration' : re.sub(r'\s{2,}',' ',duration[0]).strip(),
                'Rating' : re.sub(r'\s{2,}',' ',rating[0]).strip(),
                'Rating_count' : re.sub(r'\s{2,}',' ',rating_counts).strip(),
                'Score' : re.sub(r'\s{2,}',' ',scores[0]).strip(),
                'Ranked' : re.sub(r'\s{2,}',' ',ranked[0]).strip(),
                'Popularity' : re.sub(r'\s{2,}',' ',popularity[0]).strip(),
                'Members' : re.sub(r'\s{2,}',' ',members[0]).strip(),
                'Favorites' : re.sub(r'\s{2,}',' ',favorites[0]).strip(),
                'Synopsis' : re.sub(r'\s{2,}',' ',synopsis).strip(),
            }
            df.append(data_mal)
        return df
    

## Running the class

In [15]:
top_animes = Scrapping_top_mal('allanime',top_max = 20)
# get the data
df_top_animes = top_animes.get_var()

https://myanimelist.net/topanime.php?type=allanime&limit=0 - 200
https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood - 200
https://myanimelist.net/anime/28977/Gintama° - 200
https://myanimelist.net/anime/9253/Steins_Gate - 200
https://myanimelist.net/anime/38524/Shingeki_no_Kyojin_Season_3_Part_2 - 200
https://myanimelist.net/anime/9969/Gintama - 200
https://myanimelist.net/anime/11061/Hunter_x_Hunter_2011 - 200
https://myanimelist.net/anime/42938/Fruits_Basket__The_Final - 200
https://myanimelist.net/anime/15417/Gintama__Enchousen - 200
https://myanimelist.net/anime/39486/Gintama__The_Final - 200
https://myanimelist.net/anime/50265/Spy_x_Family - 200
https://myanimelist.net/anime/820/Ginga_Eiyuu_Densetsu - 200
https://myanimelist.net/anime/34096/Gintama - 200
https://myanimelist.net/anime/43608/Kaguya-sama_wa_Kokurasetai__Ultra_Romantic - 200
https://myanimelist.net/anime/35180/3-gatsu_no_Lion_2nd_Season - 200
https://myanimelist.net/anime/918/Gintama - 200
https://mya

In [16]:
# the output example
df_top_animes[1]

{'Title': 'Gintama°',
 'Title_alt': "Synonyms: Gintama' (2015)  Japanese: 銀魂°  English: Gintama Season 4      German: Gintama Season 4    Spanish: Gintama Temporada 4    French: Gintama Saison 4    ",
 'Link': 'https://myanimelist.net/anime/28977/Gintama°',
 'Type': 'TV',
 'Episodes': '51',
 'Aired': 'Apr 8, 2015 to Mar 30, 2016',
 'Premiered': 'Spring 2015',
 'Broadcast': 'Wednesdays at 18:00 (JST)',
 'Producers': 'TV Tokyo, Aniplex, Dentsu',
 'Licensors': 'Funimation, Crunchyroll',
 'Studios': 'Bandai Namco Pictures',
 'Source': 'Manga',
 'Genres': 'Action,Comedy,Sci-Fi',
 'Themes': 'GagHumor,Historical,Parody,Samurai',
 'Duration': '24 min. per ep.',
 'Rating': 'PG-13 - Teens 13 or older',
 'Rating_count': '214269',
 'Score': '9.081 (scored by 214269214,269 users)',
 'Ranked': '#22',
 'Popularity': '#336',
 'Members': '535,343',
 'Favorites': '14,350',
 'Synopsis': "Gintoki, Shinpachi, and Kagura return as the fun-loving but broke members of the Yorozuya team! Living in an alternate

# Convert to dataframe

In [18]:
import pandas as pd
data_animes = pd.DataFrame(df_top_animes)
data_animes.head()

,Title,Title_alt,Link,Type,Episodes,Aired,Premiered,Broadcast,Producers,Licensors,...,Themes,Duration,Rating,Rating_count,Score,Ranked,Popularity,Members,Favorites,Synopsis
0,Fullmetal Alchemist: Brotherhood,Synonyms: Hagane no Renkinjutsushi: Fullmetal ...,https://myanimelist.net/anime/5114/Fullmetal_A...,TV,64,"Apr 5, 2009 to Jul 4, 2010",Spring 2009,Sundays at 17:00 (JST),"Aniplex, Square Enix, Mainichi Broadcasting Sy...","Funimation, Aniplex of America",...,Military,24 min. per ep.,R - 17+ (violence & profanity),1839747,"9.141 (scored by 18397471,839,747 users)",#12,#3,"2,886,453","201,889",After a horrific alchemy experiment goes wrong...
1,Gintama°,Synonyms: Gintama' (2015) Japanese: 銀魂° Engl...,https://myanimelist.net/anime/28977/Gintama°,TV,51,"Apr 8, 2015 to Mar 30, 2016",Spring 2015,Wednesdays at 18:00 (JST),"TV Tokyo, Aniplex, Dentsu","Funimation, Crunchyroll",...,"GagHumor,Historical,Parody,Samurai",24 min. per ep.,PG-13 - Teens 13 or older,214269,"9.081 (scored by 214269214,269 users)",#22,#336,"535,343","14,350","Gintoki, Shinpachi, and Kagura return as the f..."
2,Steins;Gate,Japanese: STEINS;GATE English: Steins;Gate,https://myanimelist.net/anime/9253/Steins_Gate,TV,24,"Apr 6, 2011 to Sep 14, 2011",Spring 2011,Wednesdays at 02:05 (JST),"Frontier Works, Media Factory, Movic, AT-X, Ka...",Funimation,...,"Psychological,TimeTravel",24 min. per ep.,PG-13 - Teens 13 or older,1235619,"9.081 (scored by 12356191,235,619 users)",#32,#13,"2,240,077","170,950",Eccentric scientist Rintarou Okabe has a never...
3,Shingeki no Kyojin Season 3 Part 2,Japanese: 進撃の巨人 Season3 Part.2 English: Attac...,https://myanimelist.net/anime/38524/Shingeki_n...,TV,10,"Apr 29, 2019 to Jul 1, 2019",Spring 2019,Mondays at 00:10 (JST),"Production I.G, Dentsu, Mainichi Broadcasting ...",Funimation,...,"Gore,Military,Survival",23 min. per ep.,R - 17+ (violence & profanity),1299614,"9.071 (scored by 12996141,299,614 users)",#42,#29,"1,840,355","51,084",Seeking to restore humanity's diminishing hope...
4,Gintama',Synonyms: Gintama (2011) Japanese: 銀魂' Engli...,https://myanimelist.net/anime/9969/Gintama,TV,51,"Apr 4, 2011 to Mar 26, 2012",Spring 2011,Mondays at 18:00 (JST),"TV Tokyo, Aniplex, Dentsu, Trinity Sound, Mira...","None found, add some",...,"GagHumor,Historical,Parody,Samurai",24 min. per ep.,PG-13 - Teens 13 or older,207197,"9.051 (scored by 207197207,197 users)",#52,#383,"471,718","7,048","After a one-year hiatus, Shinpachi Shimura ret..."


`last step, export it into .csv/.xlsx file`